# 常用工具



## IOU

In [27]:
# 1对1的iou计算
def iou_count(bboxA, bboxB):
    # 交叉区域的坐标
    xa = max(bboxA[0], bboxB[0])
    ya = max(bboxA[1], bboxB[1])
    xb = min(bboxA[2], bboxB[2])
    yb = min(bboxA[3], bboxB[3])

    # 交叉面积计算
    inter = max(0, xb-xa+1)*max(0, yb-ya+1)  # 🍓保证两个完全重叠bbox的iou为1

    # bboxA和bboxB的面积
    areaA = (bboxA[2]-bboxA[0]+1)*(bboxA[3]-bboxA[1]+1)
    areaB = (bboxB[2]-bboxB[0]+1)*(bboxB[3]-bboxB[1]+1)
    
    # 计算iou
    iou = inter/float(areaA+areaB-inter)

    return iou

In [30]:
bboxA, bboxB = [1,1,4,4], [2,2,8,8]
print(iou_count(bboxA, bboxB))

16 49 0.16071428571428573
0.16071428571428573


## NMS

#TODO: 使用上述iou_count的一对一计算，不用numpy

In [81]:
def nms(dets, thre):
    x1, y1, x2, y2, scores = dets[:, 0], dets[:, 1], dets[:, 2], dets[:, 3], dets[:, 4]
    areas = abs(x2-x1+1)*abs(y2-y1+1)
    order = scores.argsort()[::-1]
    
    res_idx = []
    while order.size>0:
        target_idx = order[0]
        res_idx.append(target_idx)
        
        xa = np.maximum(x1[target_idx], x1[order[1:]])  # np.maximum逐位比较
        ya = np.maximum(y1[target_idx], y1[order[1:]])
        xb = np.minimum(x2[target_idx], x2[order[1:]])
        yb = np.minimum(y2[target_idx], y2[order[1:]])
        
        inter = np.maximum(0, xb-xa+1)*np.maximum(0, yb-ya+1)
        iou = inter / (areas[target_idx]+areas[order[1:]]-inter)
        
        skip_idx = np.where(iou<=thre)[0]
        print(skip_idx, order, order[skip_idx+1])
        order = order[skip_idx+1]
    
    return res_idx

In [82]:
dets = np.array([[30, 20, 230, 200, 1], 
                 [50, 50, 260, 220, 0.9],
                 [210, 30, 420, 5, 0.8],
                 [430, 280, 460, 360, 0.7]])
thresh = 0.35
keep_dets = nms(dets, thresh)

[1 2] [0 1 2 3] [2 3]
[0] [2 3] [3]
[] [3] []


In [84]:
# python3
import numpy as np

def py_nms(dets, thresh):
    # 1. 对所有dets中的点处理，计算每一个点的面积
    # x1、y1、x2、y2、以及score赋值
    x1 = dets[:, 0]
    y1 = dets[:, 1]
    x2 = dets[:, 2]
    y2 = dets[:, 3]
    scores = dets[:, 4]

    # 每一个候选框的面积
    areas = (x2 - x1 + 1) * (y2 - y1 + 1)
    #order是按照score降序排序的
    order = scores.argsort()[::-1]

    keep = []
    while order.size > 0:
        i = order[0]
        keep.append(i)
        #计算当前概率最大矩形框与其他矩形框的相交框的坐标，会用到numpy的broadcast机制，得到的是向量
        xx1 = np.maximum(x1[i], x1[order[1:]])
        yy1 = np.maximum(y1[i], y1[order[1:]])
        xx2 = np.minimum(x2[i], x2[order[1:]])
        yy2 = np.minimum(y2[i], y2[order[1:]])

        #计算相交框的面积,注意矩形框不相交时w或h算出来会是负数，用0代替
        w = np.maximum(0.0, xx2 - xx1 + 1)
        h = np.maximum(0.0, yy2 - yy1 + 1)
        inter = w * h
        #计算重叠度IOU：重叠面积/（面积1+面积2-重叠面积）
        ovr = inter / (areas[i] + areas[order[1:]] - inter)

        #找到重叠度不高于阈值的矩形框索引
        inds = np.where(ovr <= thresh)[0]
        #将order序列更新，由于前面得到的矩形框索引要比矩形框在原order序列中的索引小1，所以要把这个1加回来
        order = order[inds + 1]
    return keep

# test
if __name__ == "__main__":
    dets = np.array([[30, 20, 230, 200, 1], 
                     [50, 50, 260, 220, 0.9],
                     [210, 30, 420, 5, 0.8],
                     [430, 280, 460, 360, 0.7]])
    thresh = 0.35
    keep_dets = py_nms(dets, thresh)
    print(keep_dets)
    print(dets[keep_dets])



+++ [1 2] [0 1 2 3] [2 3]
+++ [0] [2 3] [3]
+++ [] [3] []
[0, 2, 3]
[[ 30.   20.  230.  200.    1. ]
 [210.   30.  420.    5.    0.8]
 [430.  280.  460.  360.    0.7]]
